In [1]:
import pandas as pd
import numpy as np
import re 
import itertools

def idx_to_ranges(i):
    for a, b in itertools.groupby(enumerate(i), lambda pair: pair[1] - pair[0]):
        b = list(b)
        yield b[0][1], b[-1][1]

def get_blank_regions(seq, _blankchar='~'):
    _indices = [m.start() for m in re.finditer(_blankchar,seq)]
    return list(idx_to_ranges(_indices))

# 1. Import aligned sequences and input features

In [2]:
### Import from spreadsheet ###
aligned_sequences = pd.read_csv('aligned_sequences.csv')
input_features = pd.read_csv('input_features.csv')
# display(aligned_sequences, input_features)

##### Merge back in addtl info #####
merged_df = pd.merge(aligned_sequences,input_features,on='sequence_header',how='outer',indicator=True)
# display(aligned, aligned['_merge'].value_counts())

##### Get gap windows #####
# lower aligned_seq #
merged_df['aligned_sequence'] = merged_df['aligned_sequence'].str.lower()
# Left strip blanks #
merged_df['aligned_sequence'] = merged_df['aligned_sequence'].str.lstrip('-')
# Right strip blanks #
merged_df['aligned_sequence'] = merged_df['aligned_sequence'].str.rstrip('-')
# replace '-' -> '~' #
merged_df['aligned_sequence'] = merged_df['aligned_sequence'].str.replace('-','~')
# get gap windows #
merged_df['gap_windows'] = merged_df['aligned_sequence'].apply(lambda x: get_blank_regions(x))
# replace ~ w/ 'n' #
merged_df['aligned_sequence'] = merged_df['aligned_sequence'].str.replace('~','n')

##### Keep relevant columns #####
display(merged_df)

### Inspection ###
# merged_df.to_csv('merged_df.csv')

,sequence_header,aligned_sequence,pt_code,isolate_name,sample_date,genes,country,source,arv_status,_merge,gap_windows
0,>PT1000 IS1001 Integrase naive,tttttagatgggatagataaagcccaagaagaacatgaaaaatacc...,PT1000,IS1001,4/7/2017,PR/RT,Italy,plasma,naive,both,[]
1,>PT2000 IS2001 Integrase naive,tttttagatggaatagataaggcacaagaagarcatgagaaatatc...,PT1000,IS1001,4/7/2017,Integrase,Italy,plasma,naive,both,[]
2,>PT3000 IS3001 Integrase experienced,tttttagatggaatagayaaggcccaagaagaacatgagaagtatc...,PT2000,IS2001,4/6/2016,PR/RT,Italy,plasma,naive,both,[]
3,>PT4000 IS4001 Integrase experienced,tttctagatggaatagataaggcccaagaagatcatgagaaatatc...,PT2000,IS2001,4/6/2016,Integrase,Italy,plasma,naive,both,[]
4,>PT1000 IS1001 PR/RT naive,cctcagatcactctttggcaacgacccatcgtcacagtaagggtag...,PT3000,IS3001,12/12/2012,PR/RT,Italy,plasma,Experienced,both,"[(297, 302)]"
5,>PT2000 IS2001 PR/RT naive,cctcagatcactctttggcaacgacccctcgtcacaataaagatag...,PT3000,IS3001,12/12/2012,Integrase,Italy,plasma,Experienced,both,"[(297, 302)]"
6,>PT3000 IS3001 PR/RT experienced,cctcaratcactctttggcaacgacccrtygtcacagtaagggtag...,PT4000,IS4001,9/3/2009,PR/RT,Italy,plasma,Experienced,both,"[(297, 302)]"
7,>PT4000 IS4001 PR/RT experienced,cctcaaatcactctttggcaacgacctattgtcacagtaaagatag...,PT4000,IS4001,9/3/2009,Integrase,Italy,plasma,Experienced,both,"[(297, 302)]"


# 2. Create FASTA file

In [3]:
##### Export #####
with open(f'bankit_upload.fasta', 'w') as f:
    for index, row in merged_df.iterrows():
        _sample_date = row['sample_date']
        _country = row['country']
        _pt_code = row['pt_code']
        _isolate_name = row['isolate_name']
        _genes = row['genes']
        _arv_status = row['arv_status']
        _bankit_isolate=f'{_pt_code}_{_isolate_name}_{_genes}_{_arv_status}'
        _aligned_seq = row['aligned_sequence']
        
        f.write(f'>{_bankit_isolate}')
        f.write(f'[Collection_date={_sample_date}]')
        f.write(f'[Country={_country}]')
        f.write(f'[Host=Homo sapiens]')
        f.write(f'[Organism=Human immunodeficiency virus 1]')
        f.write(f'[Isolate={_bankit_isolate}]')
        f.write(f'[Isolation source=Plasma]')
        f.write(f'\n')
        f.write(f'{_aligned_seq}\n')

# 3. Create features file

In [4]:
_stop_codon_seqs = [
    '>PT1000_IS1001_Integrase_naive',
    '>PT2000_IS2001_PR/RT_naive'
]

_misc_note_prrt = 'nonfunctional pol protein due to mutation; contains protease and reverse transcriptase'
_misc_note_in = 'nonfunctional pol protein due to mutation; contains integrase'


with open('features.txt', 'w') as f:
    for index, row in merged_df.iterrows():
        # Feature, sequence#
        _pt_code = row['pt_code']
        _isolate_name = row['isolate_name']
        _genes = row['genes']
        _arv_status = row['arv_status']
        _bankit_isolate=f'{_pt_code}_{_isolate_name}_{_genes}_{_arv_status}'
        f.write(f'>Feature {_bankit_isolate}\n')
        _end_position = len(row['aligned_sequence'])

        # gene #
        f.write(f'<1\t>{_end_position}\tgene\n')
        f.write(f'\t\t\tgene\tpol\n')

        for gap in row['gap_windows']:
            _gap_start = gap[0]
            _gap_end = gap[1]
            f.write(f'{_gap_start}\t{_gap_end}\tgap\n')
            f.write(f'\t\t\testimated_length\t{_gap_end-_gap_start+1}\n')

        if _isolate_name in _stop_codon_seqs:
            f.write(f'<1\t>{_end_position}\tmisc_feature\n')
            if row['genes'] == 'PR/RT':
                f.write(f'\t\t\tnote\t{_misc_note_prrt}\n')
            if row['genes'] == 'Integrase':
                f.write(f'\t\t\tnote\t{_misc_note_in}\n')

        # CDS #
        f.write(f'<1\t>{_end_position}\tCDS\n')
        f.write(f'\t\t\tgene\tpol\n')
        f.write(f'\t\t\tcodon_start\t1\n')
        f.write(f'\t\t\tproduct\tpol protein\n')
        f.write(f'\t\t\ttransl_table\t1\n')
        # end #
        f.write(f'\n')